# Tarea 1

Implementar la clase `OisCashflow`.

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
from datetime import date
import pandas as pd
from datetime import datetime
import math

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [4]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

AD: Todas las variables de una clase deben ser llamadas como self.nombre desde los métodos de la clase.

In [6]:
# por alguna razón las variables globales no son tomadas en cuenta
# por lo que hay que renomrbarlas en cada función como variable local

# AD: todas las variables de la clase deben ser llamadas como self.nombre

#además se crea una nueva función que recoge los datos del excel SOFR-10012019-10292020.xls

# AD: debieron cambiar el tipo de las variables fixings. Este código no pasa un type checker si es
# que se le aplicara uno.

@dataclass # syntactic sugar
class OisCashflow:
    start_date: Qcf.QCDate
    end_date: Qcf.QCDate
    settlement_date: Qcf.QCDate
    notional: float
    currency: Qcf.QCCurrency
    amortization: float
    amort_is_cashflow: bool
    interest_rate: Qcf.QCInterestRate
    on_index: Qcf.InterestRateIndex
    spread: float
    gearing: float
    filename: str
    leaf: str    
        
    def readExcel(self, filename, leaf):
        data = pd.read_excel(filename,leaf)
        #data = pd.read_excel('data/SOFR-10012019-10292020.xls','nice format')
        data['fecha_inicial']=data.DATE
        data['fecha_final']=data.DATE
        data['ratio']=1.11111
        data['ratio']=data.iloc[:,2]/100


        data['diff'] = None
        diff=data.ratio

        fixings = data.sort_values('DATE')
        for i in range(len(fixings.DATE)-1):
            x=datetime.strptime(fixings.fecha_inicial.iloc[i], '%Y-%m-%d').date()
            y=datetime.strptime(fixings.fecha_inicial.iloc[i+1], '%Y-%m-%d').date()


            fixings.fecha_inicial.iloc[i]= Qcf.QCDate(x.day, x.month, x.year)
            fixings.fecha_final.iloc[i]= Qcf.QCDate(y.day, y.month, y.year)



        fixings=fixings.iloc[0:269,3:6]
        fecha_inicial=fixings.fecha_inicial
        fecha_final=fixings.fecha_final
        valor_tasa=fixings.ratio



        def get_wf(fecha_inicial: date, fecha_final: date, valor_tasa: float) -> float:

            qc_fec_ini = fecha_inicial
    
            qc_fec_fin = fecha_final
    
            qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
            return qc_tasa.wf(qc_fec_ini, qc_fec_fin)



        fixings['wf'] = fixings.apply(
            lambda row: get_wf(row['fecha_inicial'], row['fecha_final'], row['ratio']),
            axis=1
        )
        return fixings
        
    #def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series, start_date, end_date ) -> float:
    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series ) -> float:
        
        start_date=self.start_date
        calendario = Qcf.BusinessCalendar(start_date, 1)
        accrual_date = calendario.next_busy_day(accrual_date)
        a=0
        b=0

        for i in range(len(fixings.ratio)):
            if start_date == fixings.fecha_inicial.iloc[i]:
                a=i
            if accrual_date == fixings.fecha_final.iloc[i]:
                b=i
        
        fixings=fixings.iloc[a:b+1,:]

        tasa_eq=(fixings.wf.product()-1)*360/start_date.day_diff(accrual_date)
       
        return(tasa_eq)        
        """
        Calcula la tasa equivalente desde `self.start_date` a `accrual_date`. La tasa equivalente
        se calcula como:
        
        (P - 1) * 360 / (accrual_date - self.start_date)
        
        donde P es el producto de los factores de capitalización de todas las tasas overnight
        entre `start_date` y `accrual_date`. Los valores de esas tasas deben estar almacenados
        en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        start_date=self.start_date
        calendario = Qcf.BusinessCalendar(start_date, 1)
        accrual_date = calendario.next_busy_day(accrual_date)
        diff=start_date.day_diff(accrual_date)
        
        notional=self.notional
        tasa_eq=OisCashflow.get_accrued_rate(self, accrual_date, OisCashflow.readExcel(
            self,'../data/SOFR-10012019-10292020.xls','nice format'))
        
        
        interes_flot_ois = notional * tasa_eq * diff / 360
        return(interes_flot_ois)  
        """
        Calcula el interés (plata) devengado desde `self.start_date` a `accrual_date` utilizando la tasa equivalente
        que se calcula con el método anterior.
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        
    
    def amount(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        amortization = self.amortization
        monto = OisCashflow.get_accrued_interest(self, accrual_date, OisCashflow.readExcel(
            self,'../data/SOFR-10012019-10292020.xls','nice format'))+amortization
        
        print("Flujo Total:", monto)
        
        """
        Calcula el flujo total al vencimiento (amortización más intereses devengados hasta self.end_date).
        
        Los valores de las  tasas overnight deben estar almacenados en la variable `fixings`.
        
        Ver la documentación de `QC_Financial_3` para el uso y funcionamiento de los objetos
        de tipo Qcf.time_series.
        """
        return(monto) 
    
    def test(self):
        return 'test'
    
    def test1(self, x: float):
        if x == 0:
            raise ValueError('x debe ser != 0')
        return 1 / x
    
    def test2(self, x: float):
        return self.test1(x) + 1
    
    def test3(self, x: float):
        return x + self.notiona

Construcción de un objeto `Qcf.InterestRateIndex`.

In [7]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [8]:


ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    10000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1,
    '../data/SOFR-10012019-10292020.xls',
    'nice format'
)


datos=ois.readExcel('../data/SOFR-10012019-10292020.xls','nice format')
tasa=ois.get_accrued_rate(Qcf.QCDate(1, 10, 2020), ois.readExcel('../data/SOFR-10012019-10292020.xls','nice format'))
interes=ois.get_accrued_interest(Qcf.QCDate(1, 10, 2020), ois.readExcel('../data/SOFR-10012019-10292020.xls','nice format'))
monto=ois.amount(Qcf.QCDate(1, 10, 2020), ois.readExcel('../data/SOFR-10012019-10292020.xls','nice format'))

print(f"La fecha inicial es: {ois.start_date}")
print(f"La fecha final es: {ois.start_date}")
print(f"La tasa equivalente a un año es: {tasa:.4%}")
print(f"Los interés generado es: {interes}")
print(f"el monto total al vencimiento es: {monto:.2f}")




/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Flujo Total: 10077850.899618605
La fecha inicial es: 1-10-2019
La fecha final es: 1-10-2019
La tasa equivalente a un año es: 0.7657%
Los interés generado es: 77850.89961860559
el monto total al vencimiento es: 10077850.90


In [9]:
try:
    ois.test1(0)
except Exception as e:
    print('Hacer algo')

Hacer algo


In [10]:
print(ois.start_date)

1-10-2019


In [11]:
print(ois.notional)

10000000


## Funciones

In [12]:
zcc = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
zcc=zcc.iloc[0:16,:]
fecha_inicial=datos.fecha_inicial
fecha_final=datos.fecha_final
diff=datos.ratio
zcc.plazo[4]=fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[22])
zcc.plazo[5]=fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[41])
zcc.plazo[6]=fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[62])
zcc.plazo[8]=fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[102])
zcc.plazo[11]=fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[165])
zcc.plazo[13]=fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[209])


#print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[250]))



/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launc

In [13]:
zcc.df.iloc[0]=1/datos.wf.iloc[0]

i=5
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[1]=1/datos.wf.iloc[0:i].product()

i=9
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[2]=1/datos.wf.iloc[0:i].product()

i=14
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[3]=1/datos.wf.iloc[0:i].product()

i=22
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[4]=1/datos.wf.iloc[0:i].product()

i=41
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[5]=1/datos.wf.iloc[0:i].product()

i=62
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[6]=1/datos.wf.iloc[0:i].product()

i=83
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[7]=1/datos.wf.iloc[0:i].product()

i=102
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[8]=1/datos.wf.iloc[0:i].product()

i=123
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[9]=1/datos.wf.iloc[0:i].product()

i=144
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[10]=1/datos.wf.iloc[0:i].product()

i=165
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[11]=1/datos.wf.iloc[0:i].product()

i=186
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[12]=1/datos.wf.iloc[0:i].product()

i=209
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[13]=1/datos.wf.iloc[0:i].product()

i=229
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[14]=1/datos.wf.iloc[0:i].product()

i=250
print(fecha_inicial.iloc[0].day_diff(fecha_inicial.iloc[i]))
zcc.df.iloc[15]=1/datos.wf.iloc[0:i].product()

zcc.drop(['tasa'], axis='columns', inplace=True)
zcc

7
14
21
31
62
93
125
153
182
212
244
273
307
335
365


,plazo,df
0,1,0.999948
1,7,0.999643
2,14,0.999283
3,21,0.998909
4,31,0.998400
5,62,0.997045
6,93,0.995721
7,125,0.994350
8,153,0.993125
9,182,0.992631


In [14]:
monto*zcc.df[15]

10000022.222222222

In [15]:
start_date=ois_cashflow.start_date
a=0
    for u in range(len(fixings.ratio)):
            if start_date == fixings.fecha_inicial.iloc[i]:
                a=u
    end_date=val_date
    zcc['icp']=0
    for j in len(zcc):
        if j==0
            zcc.icp.iloc[0]=fixings.ratio.iloc[a]
        else
            for i in range(len(fixings.ratio)):
                    if  j == fecha_inicial.iloc[0].day_diff(fecha_final.iloc[i]):
                        b=i
                    fixings=fixings.iloc[a:b+1,:]
            wf=fixings.wf.product()
            zcc.icp.iloc[j]=zcc.icp.iloc[0]wf
    zcc['tasa']=0
    for k in len(zcc):
        if k==0
            zcc.tasa.iloc[0]=fixings.ratio.iloc[a]
        else
            icp=zcc.icp.iloc[k]/zcc.icp.iloc[k-1]
            zcc.tasa.iloc[k] = [ICP - 1]360 / (zcc.iloc[k,0]-zcc.iloc[k-1,0])

IndentationError: unexpected indent (<ipython-input-15-61bbc752630e>, line 3)

AD: La función set_expected_rate ni siquiera se puede definir por múltiples problemas de sintaxis.
Yo hice una segunda definición sin esos errores para poder testearla.'

In [16]:
# Faltaban : al final de los if y else. Raro que no lo hayan notado.
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    start_date=ois_cashflow.start_date
    a=0
    for u in range(len(fixings.ratio)):
            if start_date == fixings.fecha_inicial.iloc[i]:
                a=u
    end_date=val_date
    zcc['icp']=0
    for j in len(zcc):
        if j==0:
            zcc.icp.iloc[0]=fixings.ratio.iloc[a]
        else:
            for i in range(len(fixings.ratio)):
                    if  j == fecha_inicial.iloc[0].day_diff(fecha_final.iloc[i]):
                        b=i
                    fixings=fixings.iloc[a:b+1,:]
            wf=fixings.wf.product()
            zcc.icp.iloc[j]=zcc.icp.iloc[0] # wf. Otro error de sintaxis grave
    zcc['tasa']=0
    for k in len(zcc):
        if k==0:
            zcc.tasa.iloc[0]=fixings.ratio.iloc[a]
        else:
            icp=zcc.icp.iloc[k]/zcc.icp.iloc[k-1]
            zcc.tasa.iloc[k] = [ICP - 1] / (zcc.iloc[k,0]-zcc.iloc[k-1,0]) # Otro más ...
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    
    # Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    # ICP(t0)*wf(to, t2) = ICP(t2), ICP(t0)*w(t0, t1)=ICP(t1) -> ICP(t2)/ICP(t1)=ICP(t1,t2)
    # tasa(t1, t2) = [ICP(t1, t2) - 1]*360 / (t2-t1)
    # El ICP se puede pensar como el account asociado a la tasa interbancaria chilena,
    # que es el índice overnight chileno.
    #
    # Cuando t1 < t0 < t2 tener un poquito de ojo ... porque el cashflow ya tiene fixings.

In [17]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OisCashflow,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    start_date=ois_cashflow.start_date
    a=0
    for u in range(len(fixings.ratio)):
        if start_date == fixings.fecha_inicial.iloc[i]:
            a=u
    end_date=val_date
    zcc['icp']=0
    for j in len(zcc):
        if j==0:
            zcc.icp.iloc[0]=fixings.ratio.iloc[a]
        else:
            for i in range(len(fixings.ratio)):
                    if  j == fecha_inicial.iloc[0].day_diff(fecha_final.iloc[i]):
                        b=i
                    fixings=fixings.iloc[a:b+1,:]
            wf=fixings.wf.product()
            zcc.icp.iloc[j]=zcc.icp.iloc[0]
    zcc['tasa']=0
    for k in len(zcc):
        if k==0:
            zcc.tasa.iloc[0]=fixings.ratio.iloc[a]
        else:
            icp=zcc.icp.iloc[k]/zcc.icp.iloc[k-1]
            zcc.tasa.iloc[k] = [ICP - 1] / (zcc.iloc[k,0]-zcc.iloc[k-1,0])
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Ver por ejemplo los casos cuando usamos el objeto Qcf.ForwardRates() en
    el notebook 9.
    """
    pass
    # Para un OIS calculando en t0 un cashflow entre (t0 <) t1 < t2
    # ICP(t0)*wf(to, t2) = ICP(t2), ICP(t0)*w(t0, t1)=ICP(t1) -> ICP(t2)/ICP(t1)=ICP(t1,t2)
    # tasa(t1, t2) = [ICP(t1, t2) - 1]*360 / (t2-t1)
    # El ICP se puede pensar como el account asociado a la tasa interbancaria chilena,
    # que es el índice overnight chileno.
    #
    # Cuando t1 < t0 < t2 tener un poquito de ojo ... porque el cashflow ya tiene fixings.

In [18]:
def present_value(val_date: Qcf.QCDate,
                  ois_cashflow: OisCashflow,
                  zcc: Qcf.ZeroCouponCurve) -> float:
    
    tenor=val_date.day_diff(Qcf.QCDate(1, 10, 2020))
    a=0
    for i in range(len(zcc.plazo)):
        if zcc.plazo[i]<=tenor+1 and zcc.plazo[i]>=tenor-1:
            a=i
    
    #vp=ois_cashflow.amount*zcc.df.iloc[a]
    vp=ois.amount(Qcf.QCDate(1, 10, 2020), ois.readExcel('../data/SOFR-10012019-10292020.xls','nice format'))*zcc.df.iloc[a]
    """
    Esta función opera de la misma forma que la análoga función de `QC_Financial_3`.
    Para probar esta función utilizar la data de que está en data/20201012_built_sofr_zero.xlsx.
    Para traer a valor presente debes respetar la convención de las tasas de la curva.
    
    NOTA: la variable ois_cashflow debe haber 'pasado' por la función set_expected_rate para que
    tenga los intereses de los fixings futuros.
    """
    return(vp)

valor_presente = present_value(
    Qcf.QCDate(1, 10, 2019),
    ois,
    zcc,
)
print(f"el valor presente es: {valor_presente}")

Flujo Total: 10077850.899618605
el valor presente es: 10000022.222222222


In [19]:
aux2=Qcf.QCDate(1, 10, 2019)
aux3=Qcf.QCDate(1, 10, 2020)
aux2.day_diff(aux3)
ois.end_date

## Tests

In [20]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [21]:
amount_tol = 10000
rate_tol = .001

In [22]:
exitos = 0

In [23]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

AD: esto está muy mal programado. El alta del objeto se hace con el nombre de la planilla con la data y la hoja que contiene los datos. Luego, en los métodos se vuelve a solicitar el nombre de la planilla.

In [24]:
ois = OisCashflow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1,
    '../data/SOFR-10012019-10292020.xls',
    'nice format'
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [25]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [26]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [27]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [28]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

AD: test modificado a solicitud el grupo. Tener en consideración que no se siguieron las instrucciones y no se explicó el cambio hecho.

In [29]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020),
                                    ois.readExcel('../data/SOFR-10012019-10292020.xls', 'nice format'))
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 1.04766527%


Se verificará el cálculo usando `df_fixings`.

In [30]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [31]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [32]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [33]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [34]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [35]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [36]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [37]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [38]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [39]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [40]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [41]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: 0.03775213%


In [42]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.accrued_interest`

In [43]:
accrued_interest = ois.get_accrued_interest(
    Qcf.QCDate(15, 6, 2020),
    ois.readExcel('../data/SOFR-10012019-10292020.xls', 'nice format'))
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 75,082.68


In [44]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [45]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [46]:
amount = ois.amount(
    ois.end_date,
    ois.readExcel('../data/SOFR-10012019-10292020.xls', 'nice format'))  # AD agrega end date
print(f'amount: {amount:,.2f}')

Flujo Total: 1077850.8996186056
amount: 1,077,850.90


In [47]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [50]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2 - 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 2823.854185


In [51]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [52]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [53]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [54]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [55]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [56]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

AttributeError: 'time_series' object has no attribute 'ratio'

In [57]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

AttributeError: 'time_series' object has no attribute 'ratio'

In [58]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

AttributeError: 'time_series' object has no attribute 'ratio'

In [59]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

AttributeError: 'time_series' object has no attribute 'ratio'

In [60]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

AttributeError: 'time_series' object has no attribute 'ratio'

### `present_value`

In [61]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

AttributeError: 'time_series' object has no attribute 'ratio'

In [62]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

AttributeError: 'time_series' object has no attribute 'ratio'

In [63]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount(fixings)
print(f'Amount: {amount:,.4f}')
amount = ois.amount(fixings)
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

AttributeError: 'time_series' object has no attribute 'ratio'

In [64]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

AttributeError: 'time_series' object has no attribute 'ratio'

In [65]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

AttributeError: 'time_series' object has no attribute 'ratio'

In [66]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 6.0
